In [1]:
%reload_ext autoreload
%autoreload 2

## Extracting the Coding Scheme

In [8]:
import requests
from bs4 import BeautifulSoup as bsoup

coding_scheme_url = 'https://middelnederlands.nl/codering/'

response = requests.get(coding_scheme_url)
response.status_code

200

In [9]:
page_soup = bsoup(response.text, 'lxml')
code_table = page_soup.find('table')
headers = [header_col.text for header_col in code_table.find('thead').find_all('th')]
headers

['Code', 'Vertaling', 'CQL']

In [10]:
code_scheme = {}

for row in code_table.find('tbody').find_all('tr'):
    row_vals = [cell.text for cell in row.find_all('td')]
    # print(row_vals)
    row_json = {header: row_vals[hi] for hi, header in enumerate(headers)}
    # print(row_json)
    code_scheme[row_json['Code']] = row_json

code_scheme

{'615': {'Code': '615',
  'Vertaling': 'PronAdv(dem)',
  'CQL': 'pos=BW, feat.form=-r/-re'},
 '645': {'Code': '645',
  'Vertaling': 'PronAdv(indef)',
  'CQL': 'pos=BW, feat.form=-r/-re'},
 '635': {'Code': '635',
  'Vertaling': 'PronAdv(inter)',
  'CQL': 'pos=BW, feat.form=-r/-re'},
 '605': {'Code': '605',
  'Vertaling': 'PronAdv(pers)',
  'CQL': 'pos=BW, feat.form=-r/-re'},
 '655': {'Code': '655',
  'Vertaling': 'PronAdv(prep)',
  'CQL': 'pos=BW, feat.form=-r/-re'},
 '625': {'Code': '625',
  'Vertaling': 'PronAdv(rel)',
  'CQL': 'pos=BW, feat.form=-r/-re'},
 '100': {'Code': '100', 'Vertaling': 'Adj()', 'CQL': 'pos=ADJ, feat.getal=ev'},
 '101': {'Code': '101',
  'Vertaling': 'Adj(forme)',
  'CQL': 'pos=ADJ, feat.getal=ev, feat.form=-e'},
 '104': {'Code': '104',
  'Vertaling': 'Adj(formn)',
  'CQL': 'pos=ADJ, feat.getal=ev, feat.form=-n'},
 '105': {'Code': '105',
  'Vertaling': 'Adj(formr)',
  'CQL': 'pos=ADJ, feat.getal=ev, feat.form=-r/-re'},
 '102': {'Code': '102',
  'Vertaling': 'Adj

In [52]:
import json

code_scheme_file = '../data/CMNL-code_scheme.json'
with open(code_scheme_file, 'wt') as fh:
    json.dump(code_scheme, fh)

In [18]:
# code_scheme_tsv_file = '../data/CMNL-code_scheme.tsv'

# with open(code_scheme_tsv_file, 'wt') as fh:
#     fh.write('\t'.join(headers) + '\n')
#     for code in sorted(code_scheme):
#         row = [code_scheme[code][header] for header in headers]
#         fh.write('\t'.join(row) + '\n')


## Parsing Corpus

In [21]:
from scripts.parse import read_code_scheme
from scripts.parse import read_docs
from scripts.parse import parse_metadata_line
from scripts.parse import parse_token_line

crm14_file = '../data/CRM14OorknrVoorElkeVorm'
crm14_file = '../data/CRM14ZESCORPORA.txt'
cgr13_file = '../data/CGR13.txt'

code_scheme_file = '../data/CMNL-code_scheme.tsv'


code_scheme = read_code_scheme(code_scheme_file)


for di, doc_lines in enumerate(read_docs(crm14_file)):
    #print(doc_lines)
    metadata = parse_metadata_line(doc_lines[0])
    for line in doc_lines[1:]:
        try:
            parse_token_line(line, code_scheme)
        except TypeError:
            print(line)
            raise
    if di > 1:
        break
    
linelen_freq

472+000 ('Art(def,forms)', 'N(sing)') ('pos=LID, feat.form=-s/-th', 'pos=N, feat.getal=ev')
700+470 ('Adp()', 'Art(def)') ('pos=VZ', 'pos=LID')
700+470 ('Adp()', 'Art(def)') ('pos=VZ', 'pos=LID')
472+002 ('Art(def,forms)', 'N(sing,forms)') ('pos=LID, feat.form=-s/-th', 'pos=N, feat.getal=ev, feat.form=-s/-th')
700+475 ('Adp()', 'Art(def,formr)') ('pos=VZ', 'pos=LID, feat.form=-r/-re')
472+002 ('Art(def,forms)', 'N(sing,forms)') ('pos=LID, feat.form=-s/-th', 'pos=N, feat.getal=ev, feat.form=-s/-th')
700+474 ('Adp()', 'Art(def,formn)') ('pos=VZ', 'pos=LID, feat.form=-n')
700+470 ('Adp()', 'Art(def)') ('pos=VZ', 'pos=LID')
+ + + &scheider; Markup(sep) - - -


ValueError: unknown pos_code: Markup(sep)